In [2]:
!pip install torch torchvision timm


# EVALUATIONS

In [1]:
from timm.layers.helpers import to_2tuple
import timm
import torch.nn as nn

class ConvStem(nn.Module):
  """Custom Patch Embed Layer.

  Adapted from https://github.com/Xiyue-Wang/TransPath/blob/main/ctran.py#L6-L44
  """

  def __init__(self, img_size=224, patch_size=4, in_chans=3, embed_dim=768, norm_layer=None, **kwargs):
    super().__init__()

    # Check input constraints
    assert patch_size == 4, "Patch size must be 4"
    assert embed_dim % 8 == 0, "Embedding dimension must be a multiple of 8"

    img_size = to_2tuple(img_size)
    patch_size = to_2tuple(patch_size)

    self.img_size = img_size
    self.patch_size = patch_size
    self.grid_size = (img_size[0] // patch_size[0], img_size[1] // patch_size[1])
    self.num_patches = self.grid_size[0] * self.grid_size[1]

    # Create stem network
    stem = []
    input_dim, output_dim = 3, embed_dim // 8
    for l in range(2):
      stem.append(nn.Conv2d(input_dim, output_dim, kernel_size=3, stride=2, padding=1, bias=False))
      stem.append(nn.BatchNorm2d(output_dim))
      stem.append(nn.ReLU(inplace=True))
      input_dim = output_dim
      output_dim *= 2
    stem.append(nn.Conv2d(input_dim, embed_dim, kernel_size=1))
    self.proj = nn.Sequential(*stem)

    # Apply normalization layer (if provided)
    self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

  def forward(self, x):
    B, C, H, W = x.shape

    # Check input image size
    assert H == self.img_size[0] and W == self.img_size[1], \
        f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."

    x = self.proj(x)
    x = x.permute(0, 2, 3, 1)  # BCHW -> BHWC
    x = self.norm(x)
    return x


In [ ]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch

# Charger l'image
img_path = "/kaggle/input/segmentation-of-nuclei-in-cryosectioned-he-images/tissue images/Human_LymphNodes_01.tif"
img = Image.open(img_path).convert("RGB")  # Convertir en RGB pour être compatible avec le modèle

# Charger le modèle préentraîné depuis Hugging Face Hub via timm
model = timm.create_model(
    model_name="hf-hub:1aurent/swin_tiny_patch4_window7_224.CTransPath",
    pretrained=True,
).eval()

# Obtenir les configurations spécifiques au modèle pour les transformations
data_config = timm.data.resolve_data_config({}, model=model)
transforms = timm.data.create_transform(**data_config, is_training=False)

# Appliquer les transformations sur l'image
data = transforms(img).unsqueeze(0)  # Ajouter la dimension batch

# Vérifier si un GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
data = data.to(device)

# Effectuer l'inférence
with torch.no_grad():
    output = model(data)

# Afficher la sortie
print("Output shape:", output.shape)
print("Output:", output)


In [16]:
import torch
import timm
from timm.models.swin_transformer import SwinTransformer
from torch import nn
from torchvision import transforms
from PIL import Image

# Define the ConvStem to match the checkpoint's architecture
class ConvStem(nn.Module):
    def __init__(self, in_channels=3, embed_dim=96):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Conv2d(in_channels, embed_dim // 2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(embed_dim // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(embed_dim // 2, embed_dim, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(embed_dim),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.proj(x)

# Create a custom Swin Transformer that uses the ConvStem
class CustomSwinTransformer(SwinTransformer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.patch_embed.proj = ConvStem(in_channels=3, embed_dim=self.embed_dim)

# Load the model with the adjusted architecture
model = CustomSwinTransformer(
    patch_size=4,
    window_size=7,
    embed_dim=96,
    depths=(2, 2, 6, 2),
    num_heads=(3, 6, 12, 24),
    num_classes=1000
)

# Load the pretrained weights
checkpoint_path = "/kaggle/input/ctranspath-models/checkpoint.pth"  # Update with the actual path
checkpoint = torch.load(checkpoint_path, map_location="cpu")
model.load_state_dict(checkpoint, strict=False)  # Load with non-strict mode to handle differences

# Move the model to GPU and set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("EVALUATIONS CI SUIT----------------------------")
model.eval()

# Define transforms
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

# Load and preprocess the image
img_path = "/kaggle/input/segmentation-of-nuclei-in-cryosectioned-he-images/tissue images/Human_LymphNodes_01.tif"
img = Image.open(img_path).convert("RGB")
data = transforms(img).unsqueeze(0).to(device)  # Add batch dimension and move to device

# Perform inference
with torch.no_grad():
    output = model(data)  # Output is (batch_size, num_features)
    print("Model output:", output)


<ipython-input-16-44732c06e71a>:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location="cpu")


EVALUATIONS CI SUIT----------------------------
Model output: tensor([[ 5.0399e-02, -2.4871e-01, -7.7308e-01, -2.8444e-02,  5.7290e-01,
         -2.0272e-01,  5.6305e-02,  2.9812e-02,  1.0317e+00, -1.2519e-02,
         -5.5127e-01,  3.1500e-01,  6.1503e-01,  1.3569e-01, -5.7848e-01,
          4.8193e-01,  5.3299e-01,  6.2942e-01,  1.5559e-02,  4.2091e-01,
          4.2335e-01, -2.8527e-01, -8.4221e-01, -5.0053e-01,  6.9965e-01,
          7.1488e-01, -1.0155e-01,  2.9721e-01,  3.1496e-01, -8.5009e-02,
          7.3454e-01, -3.0462e-01, -5.6311e-01, -2.7075e-01, -1.7599e-01,
          6.5946e-01,  1.3033e-01,  1.0066e+00, -4.3048e-01, -9.6148e-02,
          1.7295e-01,  3.0260e-01, -3.4817e-02, -3.6994e-01, -1.2041e-02,
         -9.6062e-02,  1.9331e-01, -5.4458e-02, -2.2754e-02, -1.8260e-01,
          1.0333e-01, -5.3961e-01,  9.7998e-01,  1.0315e-01,  4.6785e-01,
          3.9948e-02,  2.7503e-01,  4.0354e-01, -1.9279e-01,  4.0951e-01,
         -1.5706e+00, -3.1961e-01,  5.4199e-01, -1

# Evaluation sur le dataset CryoNuSeg

In [17]:
import os
import torch
import timm
from timm.models.swin_transformer import SwinTransformer
from torch import nn
from torchvision import transforms
from PIL import Image

# Define the ConvStem to match the checkpoint's architecture
class ConvStem(nn.Module):
    def __init__(self, in_channels=3, embed_dim=96):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Conv2d(in_channels, embed_dim // 2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(embed_dim // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(embed_dim // 2, embed_dim, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(embed_dim),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.proj(x)

# Create a custom Swin Transformer that uses the ConvStem
class CustomSwinTransformer(SwinTransformer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.patch_embed.proj = ConvStem(in_channels=3, embed_dim=self.embed_dim)

# Load the model with the adjusted architecture
model = CustomSwinTransformer(
    patch_size=4,
    window_size=7,
    embed_dim=96,
    depths=(2, 2, 6, 2),
    num_heads=(3, 6, 12, 24),
    num_classes=1000
)

# Load the pretrained weights
checkpoint_path = "/kaggle/input/ctranspath-models/checkpoint.pth"  # Update with the actual path
checkpoint = torch.load(checkpoint_path, map_location="cpu")
model.load_state_dict(checkpoint, strict=False)  # Load with non-strict mode to handle differences

# Move the model to GPU and set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Define transforms
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

# Path to the folder containing images
folder_path = "/kaggle/input/segmentation-of-nuclei-in-cryosectioned-he-images/tissue images"
results = {}

# Process all images in the folder
for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    try:
        img = Image.open(img_path).convert("RGB")  # Open and convert image to RGB
        data = transforms(img).unsqueeze(0).to(device)  # Add batch dimension and move to device
        
        # Perform inference
        with torch.no_grad():
            output = model(data)  # Output is (batch_size, num_features)
            results[img_name] = output.cpu().numpy()  # Save the output for this image
            print(f"Processed {img_name}: {output}")
    except Exception as e:
        print(f"Error processing {img_name}: {e}")

# Print results
print("Results for all images:", results)


<ipython-input-17-5207d6cbe58d>:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location="cpu")


Processed Human_Mediastinum_03.tif: tensor([[ 5.1871e-01,  1.0350e-01, -5.4259e-02, -3.6657e-01,  3.6626e-01,
          1.9395e-01, -5.1494e-01,  3.1660e-01,  3.0874e-02,  2.8180e-01,
         -1.4690e-02, -5.7027e-01, -1.0255e-01, -4.3366e-01, -3.4613e-01,
         -5.8288e-01,  2.3318e-01,  6.7877e-01, -4.7684e-01, -5.0841e-01,
          3.1102e-02, -2.9048e-02, -3.1129e-01, -1.4027e-01, -4.8320e-02,
          1.9855e-01, -8.4202e-01,  3.6843e-01,  7.1534e-01,  1.4599e-01,
          2.7523e-01, -8.5672e-01, -2.2008e-01,  6.2315e-01,  5.5682e-02,
          1.3675e-01, -8.4830e-02,  6.3717e-04, -3.9539e-01,  2.4488e-01,
          4.5109e-01,  2.6490e-01,  1.7670e-01, -5.8584e-01, -4.5491e-01,
         -6.9902e-01,  1.0058e-01, -2.7532e-01,  1.4350e-01,  3.3157e-02,
          7.4382e-01, -1.1314e+00, -2.7525e-03,  2.5424e-01, -2.6497e-01,
         -3.8809e-01, -5.0529e-01,  1.3984e-01,  8.7709e-02, -5.2015e-01,
         -1.4560e+00, -7.4208e-02, -3.0599e-02, -3.2860e-01,  1.6454e-01,
  